In [ ]:
# All necessary imports 
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Here i Define a class for ScriptData
class ScriptData:
# Here we initialize a constructor with parameters
    def __init__(self, api_key):
        self.api_key = api_key
# Here we store our dataframe 
        self.data = {}
    
# This function is used to get intraday data for 6omin interval
    def fetch_intraday_data(self, script):
        try:
            url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={script}&interval=60min&apikey={self.api_key}"
            response = requests.get(url)
            self.intraday_data = response.json()
            return self.intraday_data
        except Exception as e:
            print(f'caught {type(e)}: e')
    
# This function is used to convert intraday data into dataframe using pandas
    def convert_intraday_data(self, script):
        try:
            self.fetch_intraday_data(script)
            intraday_data = self.intraday_data["Time Series (60min)"]
            data = []
            for timestamp, values in intraday_data.items():
                data.append([timestamp, float(values["1. open"]), float(values["2. high"]), float(values["3. low"]), float(values["4. close"]), int(values["5. volume"])])
            self.df = pd.DataFrame(data, columns=["timestamp", "open", "high", "low", "close", "volume"])
            self.df['timestamp'] = pd.to_datetime(self.df['timestamp'])

            self.data[script] = self.df

            return self.df,self.data
        except Exception as e:
            print(f'caught {type(e)}: e')

In [ ]:
# This function is used to take moving average of given timeperiod with parameters as df(dataframe) and timeperiod
def indicator1(df, timeperiod):
    try:
        indicator = df["close"].rolling(timeperiod).mean()
        indicator_df = pd.DataFrame({'timestamp': df['timestamp'][1:], 'indicator': indicator})
        return indicator_df
    except Exception as e:
        print(f'caught {type(e)}: e')

In [ ]:
# Here we define a class for strategy for getting signals like buy,sell etc.
class Strategy:
    
    def __init__(self, api_key):
        self.script_data = ScriptData(api_key)
        
# This function is used to generate signals for buy and sell
    def generate_signals(self, script, timeperiod):
        try:
            self.script_data.convert_intraday_data(script)
            df = self.script_data.df
            indicator_df = indicator1(df, timeperiod)
            signals = []
# Here we check condition for buy or sell signal
            for i in range(1, len(df)):
                if indicator_df['indicator'][i] > df['close'][i] and indicator_df['indicator'][i-1] <= df['close'][i]:
                    signals.append('SELL')
                elif indicator_df['indicator'][i] < df['close'][i] and indicator_df['indicator'][i-1] >= df['close'][i-1]:
                    signals.append('BUY')
                else:
                    signals.append('NO_SIGNAL')
            self.signals = pd.DataFrame({'timestamp': df['timestamp'][1:], 'signal': signals})
            return self.signals
        except Exception as e:
            print(f'caught {type(e)}: e')


In [ ]:
api_key = "C00PZAXAW392MJA7"
script1 = "AAPL"
script2 = "GOOGL"
timeperiod = 5
script_data = ScriptData(api_key)

In [ ]:
script_data.fetch_intraday_data(script1)

In [ ]:
script_data.fetch_intraday_data(script2)

In [ ]:
script_data.convert_intraday_data(script1)
script_data.convert_intraday_data(script2)

In [ ]:
indicator1(script_data.data['AAPL'],timeperiod)

In [ ]:
script_data.data

In [ ]:
"GOOGL" in script_data.data

In [ ]:
"AAPL" in script_data.data

In [ ]:
"NVDA" in script_data.data

In [ ]:
script_data.data['AAPL']

In [ ]:
script_data.data['GOOGL']

In [ ]:
import plotly.graph_objects as go

import pandas as pd
from datetime import datetime

# df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/finance-charts-apple.csv')

fig = go.Figure(data=[go.Candlestick(x=script_data.data['AAPL']['timestamp'],
                open=script_data.data['AAPL']['open'],
                high=script_data.data['AAPL']['high'],
                low=script_data.data['AAPL']['low'],
                close=script_data.data['AAPL']['close'])])

fig.show()

In [ ]:
strategy = Strategy(api_key)

In [ ]:
strategy.generate_signals(script,timeperiod)

In [ ]:
df_ind = indicator1(script_data.data['AAPL'],timeperiod)

In [ ]:
# Check the line plot for indicator and close
# line 1 points
x1 = script_data.data['AAPL']['timestamp'].head(20)
y1 = script_data.data['AAPL']['close'].head(20)
# plotting the line 1 points 
plt.plot(x1, y1, label = "line 1")
# line 2 points
x2 = script_data.data['AAPL']['timestamp'].head(20)
y2 = df_ind['indicator'].head(20)
# plotting the line 2 points 
plt.plot(x2, y2, label = "line 2")
plt.xlabel('x - axis')
# Set the y axis label of the current axis.
plt.ylabel('y - axis')
# Set a title of the current axes.
plt.title('Two or more lines on same plot with suitable legends ')
# show a legend on the plot
plt.legend()
# Display a figure.
plt.show()